In [ ]:
pip install pandas numpy matplotlib scipy plotly

<h3>conversion.py</h3>

In [ ]:
# conversion-v5.py
import pandas as pd
from pathlib import Path

def convertir_mpd_a_csv(ruta_mpd="../data/raw",     
                        ruta_salida="../data/interim/conversion",
                        filas_saltadas=29): 
    ruta_mpd = Path(ruta_mpd) # Ruta a los .mpd
    ruta_csv = Path(ruta_salida) # Ruta donde guardar los .csv 
    ruta_csv.mkdir(exist_ok=True) # Crea la carpeta si no existe
    
    # Buscar todos los archivos .mpd
    archivos_mpd = sorted(ruta_mpd.glob("mp*.riogrande.mpd"))  
    colspecs = [
    (0, 11),     # Date (YYYY/MM/DD)
    (12, 24),    # Time (hh:mm:ss.sss)
    (25, 30),    # File ID (ej: 00RU0)
    (31, 36),    # Rge
    (37, 43),    # Ht
    (44, 51),    # Vrad
    (52, 58),    # delVr
    (59, 65),    # Theta
    (66, 73),    # Phi0
    (75, 76),    # Ambig
    (80, 85),    # Delphase
    (89, 92),    # ant-pair
    (99, 100),   # IREX
    (102, 109),  # amax
    (110, 114),  # Tau
    (115, 120),  # vmet
    (122, 127),  # snrdb
    ]
    for archivo in archivos_mpd:
        # Notificacion de progreso. 
        print(f"Convirtiendo: {archivo.name}")

        # Leer datos ignorando las primeras filas. Avisa al pandas que viene sin header
        df = pd.read_fwf(archivo, skiprows=filas_saltadas, header=None, colspecs=colspecs)

        # Convertir la columna de fecha al formato DD/MM/AAAA
        df[0] = pd.to_datetime(df[0], format="%Y/%m/%d").dt.strftime("%d/%m/%Y")

        # Nombre de archivo .csv en la nueva carpeta 
        nuevo_nombre = ruta_csv / (archivo.stem + ".csv")

        # Guardar como CSV con separador ";" y decimal "," y sin agregar header
        df.to_csv(nuevo_nombre, index=False, sep=";", decimal=",",header=False)
    
    # Confirmacion de proceso terminado. 
    print("✅ Proceso completado: todos los archivos han sido convertidos correctamente.")

convertir_mpd_a_csv()# MODIFICAR PARAMETROS SEGUN CORRESPONDA

<h2>transformacion.py</h2>

<h4>cambio el nombre de la funcion a transformacion.py</h4>

In [ ]:
import pandas as pd
from pathlib import Path

def filtro_ambig(ruta_entrada="../data/interim/conversion",
           ruta_salida="../data/interim/transformacion", 
           columna_objetivo=9, valor_permitido=1, #De la columna 9 (ambig) Elimina todos los registros distintos a 1
           columnas_a_eliminar=[2, 9, 12]): #Elimina las columnas 2 (file) 9 (ambig) y 12 (IREX)
           
    
    ruta_entrada = Path(ruta_entrada)
    ruta_salida = Path(ruta_salida)
    ruta_salida.mkdir(exist_ok=True)

    archivos_csv = sorted(ruta_entrada.glob("*.csv"))

    for archivo in archivos_csv:
        print(f"Procesando: {archivo.name}")

        # Leer el CSV sin encabezado
        df = pd.read_csv(archivo, sep=";", header=None, decimal=",")

        # Filtrar por ambigüedad
        df_filtrado = df[df[columna_objetivo] == valor_permitido]

        # Eliminar columnas especificadas
        df_resultado = df_filtrado.drop(columns=columnas_a_eliminar)

        # Guardar resultado
        nuevo_nombre = ruta_salida / archivo.name
        df_resultado.to_csv(nuevo_nombre, index=False, sep=";", decimal=",", header=False)

    print("✅ Proceso de filtrado y depuración completado correctamente.")
filtro_ambig()

<h2>Filtrado por RGE 100-130</h2>

In [ ]:
import pandas as pd
from pathlib import Path

def filtro_rge(ruta_entrada="../data/interim/transformacion",
                   ruta_salida="../data/interim/filtrado",
                   indice_rge=2,
                   rge_min=100,
                   rge_max=130):
    
    ruta_entrada = Path(ruta_entrada)
    ruta_salida = Path(ruta_salida)
    ruta_salida.mkdir(exist_ok=True)

    archivos_csv = sorted(ruta_entrada.glob("*.csv"))

    for archivo in archivos_csv:
        print(f"Filtrando por Rge: {archivo.name}")

        # Leer archivo filtrado previamente
        df = pd.read_csv(archivo, sep=";", header=None, decimal=",")

        # Aplicar filtro por Rge entre 100 y 130 inclusive
        df_filtrado = df[(df[indice_rge] >= rge_min) & (df[indice_rge] <= rge_max)]

        # Guardar archivo filtrado
        nuevo_nombre = ruta_salida / archivo.name
        df_filtrado.to_csv(nuevo_nombre, index=False, sep=";", decimal=",", header=False)

    print("✅ Filtrado por Rge completado correctamente.")

filtro_rge()


<h2>Transformacion.py</h2>
<h4>Elimino la funcion transformacion.py (No vamos a usar el valor amaxT y reordenamos la funcion)</h4>

In [ ]:
# #transformacion.py
# import pandas as pd
# from pathlib import Path

# def transformar_archivos_csv(
#         ruta_entrada="INDICE_DE_MASA/csv/filtrado/rge",
#         ruta_salida="INDICE_DE_MASA/csv/transformado"):
#     ruta_entrada = Path(ruta_entrada)
#     ruta_salida = Path(ruta_salida)
#     ruta_salida.mkdir(exist_ok=True)

#     archivos_csv = sorted(ruta_entrada.glob("*.csv"))

#     for archivo in archivos_csv:
#         print(f"Procesando: {archivo.name}")
        
#         # Cargar el CSV sin encabezado, usando ; como separador y , como decimal
#         df = pd.read_csv(archivo, sep=";", decimal=",", header=None)

#         # Verificar que haya al menos 14 columnas
#         if df.shape[1] < 14:
#             print(f"⚠️ {archivo.name} tiene menos de 14 columnas. Se omite.")
#             continue

#         # Nueva columna: columna 10 - columna 13 (índices)
#         nueva_columna = df.iloc[:, 10] - df.iloc[:, 13]
        
#         # Agregar la nueva columna al final
#         df[df.shape[1]] = nueva_columna

#         # Guardar el nuevo archivo, sin encabezado
#         salida = ruta_salida / archivo.name
#         df.to_csv(salida, sep=";", decimal=",", index=False, header=False)
    
#     print("✅ Transformación completa.")

# transformar_archivos_csv()


<h2>consolidacion.py</h2>
<h4>modifique las rutas.
elimino la columna antes agregada de amaxT (amax - snrdb)</h4>

In [ ]:
# consolidacion.py
import pandas as pd
from pathlib import Path

def consolidar_csv(ruta_entrada="../data/interim/filtrado", ruta_salida="../data/processed/2024_consolidado.csv"): # Rutas predeterminadas
    ruta_entrada = Path(ruta_entrada)
    # Buscar todos los archivos con extensión .csv dentro de esa carpeta
    archivos_csv = sorted(ruta_entrada.glob("*.csv"))
    
    # ruta_salida = Path(ruta_salida)
    # ruta_salida.mkdir(exist_ok=True)
    
    # Lista de nombres de columnas, ya que los archivos CSV individuales no tienen encabezado
    columnas = [
        "Date", "Time", "Rge", "Ht", "Vrad", "delVr",
        "Theta", "Phi0", "Delphase", "ant-pair",
        "amax", "Tau", "vmet", "snrdb"
    ]
    
    # Lista vacía para acumular los DataFrames de cada archivo
    dataframes = []
    
    # Iterar sobre cada archivo CSV individual
    for archivo in archivos_csv:
        # Notificacion de progreso.
        print(f"Leyendo: {archivo.name}")

        # Leer el archivo sin encabezado, asigna los nombres de columnas definidas arriba y usando ; como separador y , como decimal
        df = pd.read_csv(archivo, header=None, names=columnas, sep=";", decimal=",")
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)

    # Unir todos los DataFrames
    df_total = pd.concat(dataframes, ignore_index=True)

   
    # Guardar
    df_total.to_csv(ruta_salida, index=False, sep=";",decimal=",")
    
    # Mensaje de confirmación al finalizar
    print("✅ Consolidación completada sin errores de columnas.")

consolidar_csv() # MODIFICAR PARAMETROS SEGUN CORRESPONDA


<h2>elimino todo salvo amax</h2>
se modifico la funcion para elimnar todo, salvo amax  (en lugar de amaxt)

In [1]:
import pandas as pd

def dejar_solo_ultima_columna(ruta_entrada, ruta_salida):
    # Leer el CSV original (con encabezado, separador ; y decimal ,)
    df = pd.read_csv(ruta_entrada, sep=";", decimal=",")

    # Seleccionar solo la última columna (por posición)
    ultima_columna = df.iloc[:, [10]]  # [-1] mantiene el DataFrame en lugar de Series

    # Guardar sin encabezado y sin índice
    ultima_columna.to_csv(ruta_salida, sep=";", decimal=",", index=False, header=False)

    print("✅ Archivo guardado solo con la última columna.")

# Ejemplo de uso
dejar_solo_ultima_columna("../data/processed/2024_consolidado.csv", "../data/processed/2024_consolidado_amax.txt")

✅ Archivo guardado solo con la última columna.
